In [5]:
import numpy as np
import pandas as pd
from collections import Counter

In [23]:
df = pd.read_csv("data/train.csv")
df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15672983.0,Ives,671.0,Germany,Male,33.0,8.0,102994.04,1.0,1.0,1.0,176471.78,0.0
1,1,15617304.0,Ojiofor,663.0,France,Male,29.0,7.0,0.00,2.0,0.0,0.0,161807.17,0.0
2,2,15787026.0,Lai,710.0,France,Male,32.0,1.0,0.00,2.0,1.0,0.0,44001.11,0.0
3,3,15593365.0,Yobachi,686.0,France,Male,52.0,9.0,82596.80,1.0,0.0,0.0,125111.71,0.0
4,4,15693637.0,Rizzo,675.0,France,Male,42.0,6.0,129968.67,1.0,1.0,1.0,70563.90,1.0


In [24]:
df_dropped = df.drop(columns=["id", "CustomerId", "Surname", "Geography"])
df_dropped.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,671.0,Male,33.0,8.0,102994.04,1.0,1.0,1.0,176471.78,0.0
1,663.0,Male,29.0,7.0,0.00,2.0,0.0,0.0,161807.17,0.0
2,710.0,Male,32.0,1.0,0.00,2.0,1.0,0.0,44001.11,0.0
3,686.0,Male,52.0,9.0,82596.80,1.0,0.0,0.0,125111.71,0.0
4,675.0,Male,42.0,6.0,129968.67,1.0,1.0,1.0,70563.90,1.0


In [26]:
df_encoded = pd.get_dummies(df_dropped, columns=["Gender"], dtype=int)
df_encoded.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Female,Gender_Male
0,671.0,33.0,8.0,102994.04,1.0,1.0,1.0,176471.78,0.0,0,1
1,663.0,29.0,7.0,0.00,2.0,0.0,0.0,161807.17,0.0,0,1
2,710.0,32.0,1.0,0.00,2.0,1.0,0.0,44001.11,0.0,0,1
3,686.0,52.0,9.0,82596.80,1.0,0.0,0.0,125111.71,0.0,0,1
4,675.0,42.0,6.0,129968.67,1.0,1.0,1.0,70563.90,1.0,0,1


In [27]:
df_encoded["Exited"].to_numpy(dtype=int)

array([0, 0, 0, ..., 1, 0, 1])

In [6]:
# Define the KNN class
class KNN:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        # TODO: Implement the fit method
        self.X = X
        self.y = y
        pass

    def predict(self, X):
        # TODO: Implement the predict method

        predictions = []

        for row in X:
            if row in self.X:
                predictions.append([self.Y[i] for i in range(len(self.Y)) if self.X[i] == row][0])
            else:
                #distances = [[self.compute_distance(row, self.X[i]), self.Y[i]] for i in range(len(self.X))]
                #orted_dis = sorted(distances)
                #topk = sorted_dis[:self.k]

                #classification = self.neighbor_majority(topk)

                #predictions.append(classification)
                distances = np.apply_along_axis(self.compute_distance, 1, self.X, row)
                dis_classifications = np.concatenate((distances, self.Y), axis=1)
                sort_dis_class = np.sort(dis_classifications)

                topk = sort_dis_class[:self.k]
                classification = self.neighbor_majority(topk)

                predictions.append(classification)

        return predictions

    def neighbor_majority(self, topk):
        #class_counts = Counter([classification for _, classification in topk])

        #most_frequent_classes = class_counts.most_common()

        #if len(most_frequent_classes) == 1:
        #    return most_frequent_classes[0][0]
        #else:
        #    if most_frequent_classes[0][1] > most_frequent_classes[1][1]:
        #        return most_frequent_classes[0][0]
        #    else:
        #        inv_sq_sums = {0:0, 1:0}
        #        for dis, classification in topk.items():
        #            inv_sq_sums[classification] += 1 / (dis ** 2)
        #            return 1 if inv_sq_sums[1] > inv_sq_sums[0] else 0
        class_counts = {1:0, 0:0}
        inv_sq_sums = {1:0, 0:0}
        for dis, classification in topk:
            class_counts[classification] += classification
            inv_sq_sums[classification] += 1 / (dis ** 2)

        if class_counts[1] != class_counts[0]:
            return 1 if class_counts[1] > class_counts[0] else 0
        else:
            return 1 if inv_sq_sums[1] > inv_sq_sums[0] else 0
        


        

    def compute_distance(self, X1, X2):
        # TODO: Implement distance computation based on self.distance_metric
        # Hint: Use numpy operations for efficient computation

        return np.linalg.norm(X1 - X2)

In [6]:
# Define data preprocessing function
def preprocess_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    labels = train_data["Exited"].to_numpy(dtype=int)
    
    train_dropped = train_data.drop(columns=["id", "CustomerId", "Surname", "Geography", "Exited"])
    test_dropped = test_data.drop(columns=["id", "CustomerId", "Surname", "Geography", "Exited"])

    train_encoded = pd.get_dummies(train_dropped, columns=["Gender"], dtype=int)
    test_encoded = pd.get_dummies(test_dropped, columns=["Gender"], dtype=int)

    # TODO: Implement data preprocessing
    # Handle categorical variables, scale features, etc.
    return _, labels, _

In [7]:
# Define cross-validation function
def cross_validate(X, y, knn, n_splits=5):
    # TODO: Implement cross-validation
    # Compute ROC AUC scores
    pass

In [ ]:
# Load and preprocess data
X, y, X_test = preprocess_data('/path/of/train.csv', '/path/of/test.csv')

# Create and evaluate model
knn = KNN(k=5, distance_metric='euclidean')

# Perform cross-validation
cv_scores = cross_validate(X, y, knn)

print("Cross-validation scores:", cv_scores)

# TODO: hyperparamters tuning


# TODO: Train on full dataset with optimal hyperparameters and make predictions on test set
knn = ...
knn.fit(X, y)
test_predictions = knn.predict(X_test)

# Save test predictions
pd.DataFrame({'id': pd.read_csv('/path/of/test.csv')['id'], 'Exited': test_predictions}).to_csv('submissions.csv', index=False)